# <b>House Prices - Polars & RandomForestRegressor</b>
Competition information may be found on the following link: [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview). Bare in mind that the main motivation behind the usage of polars comes from the following articles in which it is stated how huge of an improvement it supposes in terms of performance against other frameworks such as Pandas: [Database-like ops benchmark](https://h2oai.github.io/db-benchmark/)

The outcome of this notebook supposes a top 40% score at the moment of edition.

## Imports & configuration

In [69]:
import time
import numpy as np
import pandas as pd
import polars as pl
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error

In [2]:
pl.Config.set_tbl_rows(100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read train dataframe

In [56]:
def data_preparation(csv_file: str):
    # Load dataframe
    df = pl.from_pandas(pd.read_csv(csv_file))

    # Remove columns with a high level of nulls and columns with a lower sample coverage than 80%
    df = df.drop(
        ["MiscFeature", "Fence", "PoolQC", "Alley", "FireplaceQu"] + ['MSZoning', 'Utilities']
    )
    
    # NULL FILLING STRATEGY
    ## LotFrontage: data_description.txt indicates NAs mean no frontage, i.e. 0.0 area 
    df = df.with_columns(
        LotFrontage=pl.col("LotFrontage").fill_null(0.0),)
    
    ## MasVnrArea/MasVnrType: relation between area and type allow us to extract the missing values, 
    ## since it means we do not have those values
    df = df.with_columns(
        MasVnrArea=pl.when(
            (pl.col("MasVnrType") == "None") & (pl.col("MasVnrArea") != 0.0)).then(pl.lit(0.0))
        .otherwise(pl.col("MasVnrArea")).fill_null(pl.lit("0.0")),
        MasVnrType=pl.when(
            (pl.col("MasVnrType") != "None") & (pl.col("MasVnrArea") == 0.0)).then(pl.lit("None"))
        .otherwise(pl.col("MasVnrType")).fill_null(pl.lit("None")),)
    
    ## BsmtExposure/BsmtFinType2/Electrical: fill utilising mode
    df = df.with_columns(
        BsmtExposure=pl.when(
            (pl.col("BsmtExposure").is_null()) & (~pl.col("BsmtCond").is_null())).then(pl.col("BsmtExposure").mode())
        .otherwise(pl.col("BsmtExposure")).fill_null("NA"),
        BsmtFinType2=pl.when(
            (pl.col("BsmtFinType2").is_null()) & (pl.col("BsmtFinSF2") != 0)).then(pl.col("BsmtFinType2").mode())
        .otherwise(pl.col("BsmtFinType2")).fill_null("NA"),
        Electrical=pl.col("Electrical").fill_null(pl.col("Electrical").mode()),)

    ## Bsmt & Garage: unknown information in small amounts
    df = df.with_columns(
        BsmtQual=pl.col("BsmtQual").fill_null("NA"),
        BsmtCond=pl.col("BsmtCond").fill_null("NA"),
        BsmtFinType1=pl.col("BsmtFinType1").fill_null("NA"),
        GarageType=(pl.col("GarageType").fill_null(pl.lit("NA"))),
        GarageYrBlt=(pl.col("GarageYrBlt").fill_null(pl.lit("NA"))),
        GarageFinish=(pl.col("GarageFinish").fill_null(pl.lit("NA"))),
        GarageQual=(pl.col("GarageQual").fill_null(pl.lit("NA"))),
        GarageCond=(pl.col("GarageCond").fill_null(pl.lit("NA"))),
    )

    # NONLINEAR TRANSFORMS OVER NUMERICAL COLUMNS
    ## Logarithm and square root of numerical columns
    NUMERIC_POLARS_DTYPES = [
        pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64, 
    ]
    num_cols = [col for col in df.select(pl.col(NUMERIC_POLARS_DTYPES)).columns if col != "SalePrice"]
    for col in num_cols:
        df = df.with_columns(
            np.log(pl.col(col)).alias(col + "_log"),
            np.sqrt(pl.col(col)).alias(col + "_sqrt"))

    # CATEGORICAL ENCODING
    ## Assign categories with relatable numerical correspondance
    ## ExterQual, ExterCond, BsmtQual, BsmtCond, HeatingQC, KitchenQual, GarageQual, GarageCond
    mapper_quality = {
        'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5,
    }

    ## BsmtExposure
    mapper_exposure = {
        'NA': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4,
    }

    ## CentralAir
    mapper_centair = {
        'N': 0, 'Y': 1
    }

    ## Electrical
    mapper_elec = {
        'FuseA': 1, 'FuseF': 1, 'FuseP': 1, 'Mix': 1,
        'SBrkr': 2
    }

    df = df.with_columns(
        ExterQual_num=pl.col("ExterQual").map_dict(mapper_quality),
        ExterCond_num=pl.col("ExterCond").map_dict(mapper_quality),
        BsmtQual_num=pl.col("BsmtQual").map_dict(mapper_quality),
        BsmtCond_num=pl.col("BsmtCond").map_dict(mapper_quality),
        HeatingQC_num=pl.col("HeatingQC").map_dict(mapper_quality),
        KitchenQual_num=pl.col("KitchenQual").map_dict(mapper_quality),
        GarageQual_num=pl.col("GarageQual").map_dict(mapper_quality),
        GarageCond_num=pl.col("GarageCond").map_dict(mapper_quality),
        BsmtExposure_num=pl.col("BsmtExposure").map_dict(mapper_exposure),
        CentralAir_num=pl.col("CentralAir").map_dict(mapper_centair),
        Electrical_num=pl.col("Electrical").map_dict(mapper_elec),
    ).drop(
        "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC", 
        "KitchenQual", "GarageQual", "GarageCond",
        "BsmtExposure", "CentralAir", "Electrical"
    )
    
    # FEATURE ENGINEERING
    ## - Remod: Whether it has been remodelated or not
    ## - RemodConst2Sold: Define category from years since remod or construction to being sold
    ##       4: 0-9, 3: 10-24, 2: 25-49, 1: 50-inf
    ## - Floors
    ## - TotalLivingSF: Total square feet of living area
    ## - ExtrasSF: Total extras square feet
    ## - Garage, WoodDeck, OpenPorch, HasEnclosedPorch, ThreeSsnPorch, HasScreenPorch, Pool
    ## Whether there is any of the above mentioned amenities
    ## - Bathrooms: Aggregated volume of bathrooms
    ## - SurelyRich: Information whether the house is sold on high prices if it has either:
    ##    special heating, screenPorch or if MiscVal > 1000
    ## - Livability: Livability score, qual * cond
    df = df.with_columns(
        Remod=pl.when(
            (pl.col("YearBuilt") == pl.col("YearRemodAdd"))).then(pl.lit(0))
        .otherwise(pl.lit(1)),
        RemodConst2Sold=pl.when(
            (pl.col("YrSold") - pl.col("YearRemodAdd")) >= 50).then(pl.lit(1))
        .when(
            ((pl.col("YrSold") - pl.col("YearRemodAdd")) < 50) &
            ((pl.col("YrSold") - pl.col("YearRemodAdd")) >= 25)).then(pl.lit(2))
        .when(
            ((pl.col("YrSold") - pl.col("YearRemodAdd")) < 25) &
            ((pl.col("YrSold") - pl.col("YearRemodAdd")) >= 10)).then(pl.lit(3))
        .when((pl.col("YrSold") - pl.col("YearRemodAdd")) < 10).then(pl.lit(4)),
        Floors=pl.when((pl.col("1stFlrSF") > 0) & (pl.col("2ndFlrSF") > 0)).then(pl.lit(3))
            .when((pl.col("1stFlrSF") > 0) & (pl.col("2ndFlrSF") == 0)).then(pl.lit(2))
            .when((pl.col("1stFlrSF") == 0) & (pl.col("2ndFlrSF") == 0)).then(pl.lit(1)),
        TotalLivingSF=pl.col("GrLivArea") + pl.col("TotalBsmtSF"),
        ExtrasSF=pl.col("GarageArea") + pl.col("WoodDeckSF") + pl.col("OpenPorchSF") + pl.col("EnclosedPorch") + pl.col("3SsnPorch") + pl.col("ScreenPorch") + pl.col("PoolArea"),
        Garage=pl.when((pl.col("GarageArea") > 0)).then(1).otherwise(0),
        WoodDeck=pl.when((pl.col("WoodDeckSF") > 0)).then(1).otherwise(0),
        OpenPorch=pl.when((pl.col("OpenPorchSF") > 0)).then(1).otherwise(0),
        HasEnclosedPorch=pl.when((pl.col("EnclosedPorch") > 0)).then(1).otherwise(0),
        ThreeSsnPorch=pl.when((pl.col("3SsnPorch") > 0)).then(1).otherwise(0),
        HasScreenPorch=pl.when((pl.col("ScreenPorch") > 0)).then(1).otherwise(0),
        Pool=pl.when((pl.col("PoolArea") > 0)).then(1).otherwise(0),
        Bathrooms=pl.col("BsmtFullBath") + pl.col("FullBath") + 0.5 * pl.col("BsmtHalfBath") + 0.5 * pl.col("BsmtHalfBath"),
        SurelyRich=pl.when(
            (pl.col("ScreenPorch") > 0) |
            (pl.col("PoolArea") > 0) |
            (pl.col("MiscVal") > 3000) |
            (pl.col("Heating") == "Floor")).then(1)
        .otherwise(0),
        Livability=pl.col("OverallCond") * pl.col("OverallQual")
    )

    # ONE HOT ENCODDING OVER CATEGORICAL COLUMNS
    CATEGORICAL_POLARS_DTYPES = [pl.Utf8]
    cat_columns = pl.col(CATEGORICAL_POLARS_DTYPES)
    columns_get_dummies = df.select(cat_columns).columns
    df = df.to_dummies(columns=columns_get_dummies)
    
    # For the rest of missing values, using forward value
    return df.fill_null(strategy="forward")

In [65]:
train_df = data_preparation("../data/train.csv")
test_df = data_preparation("../data/test.csv")

/Users/lridrue1/opt/anaconda3/envs/pproj310/lib/python3.10/site-packages/polars/series/series.py:1113: RuntimeWarning: divide by zero encountered in log
  series = f(lambda out: ufunc(*args, out=out, dtype=dtype_char, **kwargs))
/Users/lridrue1/opt/anaconda3/envs/pproj310/lib/python3.10/site-packages/polars/series/series.py:1113: RuntimeWarning: divide by zero encountered in log
  series = f(lambda out: ufunc(*args, out=out, dtype=dtype_char, **kwargs))


In [66]:
def feature_selection(df: pl.dataframe, features=[]):
    # Transform polars dataframe to pandas
    pd_df = df.to_pandas().replace([np.inf, -np.inf], 0, inplace=False)
    if not features:
        # Split onto X/y
        X = pd_df.drop(columns=["SalePrice"])
        y = pd_df[["SalePrice"]]

        # Obtain feature selection overfitting DecisionTreeClassifier
        selector = SelectFromModel(estimator=DecisionTreeClassifier(random_state=2023))
        selector = selector.fit(X.dropna(), np.ravel(y))

        return pd_df[X.columns[selector.get_support()].tolist() + ["SalePrice"]], X.columns[selector.get_support()].tolist()
    else:
        return pd_df[features]


In [67]:
pd_train, train_features = feature_selection(train_df)
pd_test = feature_selection(test_df, train_features)

In [87]:
# Due to possible missing columns after applying OHE on test, we fill non-existent values and drop missing from train
pd_test = pd_test.reindex(columns=list(set().union(pd_train.columns, pd_test.columns) - set(["SalePrice"])), fill_value=0) \
    .drop(columns=list(set(pd_test.columns) - set(pd_train.columns)))

## Model

In [70]:
# Split the data onto features and target
X = pd_train.drop(columns=["SalePrice"])
y = pd_train["SalePrice"]

# test_size=0.2 variable means that 20% of the data will be utilised for tests and 
# random_state=42 is a seed to be able to reproduce results on every iteration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the hyperparameters for the Random Forest Regressor GridSearch
params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [2, 5, 7, 10],
    'min_samples_leaf': [1, 5, 10, 20],
}

# Create a Random Forest Regressor model
rf = RandomForestRegressor()

def score_func(y_true, y_pred, **kwargs):
    return -mean_squared_log_error(y_true, y_pred, squared=False)

scorer = make_scorer(score_func)

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(rf, params, cv=6, scoring=scorer)
grid_search.fit(X_train, y_train)

# Evaluate the best model using cross-validation
best_model = grid_search.best_estimator_
best_score = np.abs(grid_search.best_score_)
print("[Train] - Kaggle Metric - Rooted Mean Squared Log Error:", best_score)

# Predict on the test data using the best estimator
y_pred = best_model.predict(X_test)

# Testing validation
msle = mean_squared_log_error(y_test, y_pred, squared=False)
print("[Test] - Kaggle Metric - Rooted Mean Squared Log Error:", msle)

[Train] - Kaggle Metric - Rooted Mean Squared Log Error: 0.14338099296115728
[Test] - Kaggle Metric - Rooted Mean Squared Log Error: 0.13680596062067443


In [94]:
# Predict on test_df
out = pd.DataFrame(
    best_model.predict(pd_test[[col for col in pd_train.columns.tolist() if col != "SalePrice"]]), 
    columns = ['SalePrice']
)
out["Id"] = pd.Series(list(range(1461, 2922)))

In [96]:
out.to_csv("../submissions/[RandomForest] polars-house_prices.csv", header=True, index=False)